In [4]:
!pip install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Running setup.py clean for scikit-surprise
Failed to build scikit-surprise
    Running setup.py install for scikit-surprise: started
    Running setup.py install for scikit-surprise: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\ranji\.matplotlib\ana\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\ranji\\AppData\\Local\\Temp\\pip-install-x93pnr3z\\scikit-surprise_060958e01a4b4efeaafe9efffe76ba62\\setup.py'"'"'; __file__='"'"'C:\\Users\\ranji\\AppData\\Local\\Temp\\pip-install-x93pnr3z\\scikit-surprise_060958e01a4b4efeaafe9efffe76ba62\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\ranji\AppData\Local\Temp\pip-wheel-rs1e9s5c'
       cwd: C:\Users\ranji\AppData\Local\Temp\pip-install-x93pnr3z\scikit-surprise_060958e01a4b4efeaafe9efffe76ba62\
  Complete output (50 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating

In [5]:
import numpy as np
import pandas as pd
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'surprise'

## __1 - Business Problem__  
___Recommend a best book based on the ratings___  

## __2 - Data collection and description__ 

In [ ]:
df = pd.read_csv("C:/Users/SohailShaikh/Downloads/book_r.csv", encoding = "ISO-8859-1")

In [ ]:
df1 = df.iloc[:,1:]

In [ ]:
df1.columns = ['userID', 'title', 'bookRating']

In [ ]:
df1.head()

In [ ]:
print('# of records: %d\n# of books: %d\n# of users: %d' % (len(df1), len(df1['title'].unique()), len(df1['userID'].unique())))

In [ ]:
palette = sns.color_palette("RdBu", 10)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.countplot(x='bookRating', data=df1, palette=palette)
ax.set_title('Distribution of book ratings')

plt.show()

### The majority of ratings is between 5 and 10. Most often users tend to rate books for 8. Second the most frequent score is 7.

## __3 - Introduction to "Surprice" Package__ 

In [ ]:
df1.bookRating.unique()

In [ ]:
reader = surprise.Reader(rating_scale=(1, 10))

In [ ]:
data = surprise.Dataset.load_from_df(df1[['userID', 'title', 'bookRating']], reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.20)

## __4 - Finding the best algorithm for our Recommendation System__ 

In [ ]:
benchmark = []

In [ ]:
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), BaselineOnly(), CoClustering()]:
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

In [ ]:
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

### SVD () algorithm gave us the best rmse, therefore, we will train and predict with SVD

## __5 - Building our Recommendation System using surprice__ 

In [ ]:
algo = SVD()

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
accuracy.rmse(predictions, verbose=True)

In [ ]:
def recommendation(userID):
    user = df1.copy()
    already_read = df1[df1['userID'] == userID]['title'].unique()
    user = user.reset_index()
    user = user[~user['title'].isin(already_read)]
    user['Estimate_Score']=user['title'].apply(lambda x: algo.predict(userID, x).est)
    #user = user.drop('title', axis = 1)
    user = user.sort_values('Estimate_Score', ascending=False)
    print(user.head(10))

## __6 - Building our Recommendation System using correlation__ 

In [ ]:
rating = pd.pivot_table(df1, index='userID', values='bookRating', columns='title', fill_value=0)

In [ ]:
corr = np.corrcoef(each_book_rating.T)

In [ ]:
corr.shape

In [ ]:
book_list=  list(rating)

In [ ]:
book_titles =[] 

In [ ]:
for i in range(len(book_list)):
    book_titles.append(book_list[i])

In [ ]:
book_titles

In [ ]:
def get_recommendation(books_list):
    book_similarities = np.zeros(corr.shape[0])
    
    for book in books_list:
        book_index = book_titles.index(book)
        book_similarities += corr[book_index] 
        book_preferences = []
    for i in range(len(book_titles)):
        book_preferences.append((book_titles[i],book_similarities[i]))
        
    return sorted(book_preferences, key= lambda x: x[1], reverse=True)

## __7 - Conclusion__ 

In [ ]:
recommendation(276747)

### You can enter any User ID and get the recommendation and estimated score

In [ ]:
my_fav_books = ['Classical Mythology','Clara Callan']

In [ ]:
print('The books you might like: \n' , get_recommendation(my_fav_books)[:10])

### You can enter your favourite book and get the recommendation on what might like